# Steps to Update Reinsurance Groupings text file
1. Check to see if last Treaties added to Ceded Register are in text file.
2. If so, then generate text file for finance, else move to step 3
3. Inform Outward about new treaties
4. Once they updated their information and provided a Grouping ID, update text file

# After Reinsurance Grouping txt file is updated

Create Excel file with format for Finance

In [ ]:
import sys, os
# Need to add parent folder to SYS PATH in order to import packages.
sys.path.insert(0, os.path.abspath('..'))

from inwards_tasks.treaty_register import Register

import pandas as pd
import numpy as np

In [ ]:
from inwards_tasks.treaty_register import Register

trg = Register(
    filepath='Assumed Non Group Treaty Register Report UWY 2007 - 2022 March 31st as at 01.04.2022.xlsx',
    report_date='31/03/2022',
)

In [ ]:
file_path = '202204XXHHMM_2022Q1_IRE_REINSURANCEGROUPINGS_v02.txt'
df = pd.read_csv(file_path, sep='\t')

df.dropna(subset=['PARTNER_GROUPING_ID'], inplace=True)

df.rename(columns={'PARTNER_GROUPING_ID': 'Grouping ID'}, inplace=True)

# Remove UWY less than 10
# If UWY is 9 or lower, CONTRACT_ID will have 11 digits because
# the year is one digit
df = df.loc[df['CONTRACT_ID'].str.len() == 12]

In [ ]:
reg_cols = ['Comp',
            'Seq',
            'UW Yr',
            'CONTRACT_ID',
            'Pd Beg',
            'Pd End',
            'Retro Seq']

df_merged = df.merge(trg.register[reg_cols], on='CONTRACT_ID', how='left')
df_merged.rename(columns={'Comp': 'COMP',
                          'Seq': 'SEQ',
                          'UW Yr': 'UWY',
                          'CONTRACT_ID': 'Sol II ID',
                          'Pd Beg': 'Start Date',
                          'Pd End': 'End Date',
                          'Retro Seq': 'Balloon Ref'},
                 inplace=True)

df_merged['COMP'] = df_merged['COMP'].astype(np.int64)
df_merged['Grouping ID'] = df_merged['Grouping ID'].astype(np.int64)
df_merged['Grouping ID'] = df_merged['Grouping ID'].astype(np.int64)
df_merged['Balloon Ref'] = df_merged['Balloon Ref'] + \
    df_merged['UWY'].astype(str)

df_merged[['COMP',
           'SEQ',
           'UWY',
           'Sol II ID',
           'Grouping ID',
           'Start Date',
           'End Date',
           'Balloon Ref']].to_clipboard(index=False)